In [3]:
%load_ext autoreload
%autoreload 2

import sys
import time
from functools import partial

# to get the ray dashboard, use `pip install 'ray[default]'`
import ray

sys.path.append('../')
from vflow import Vset, Vfunc, AsyncModule, init_args

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# initialize ray
ray.init(num_cpus=2) # optionally might add _temp_dir='tmp'

In [3]:
def slow_fun(a, b=1):
    time.sleep(1)
    return a + b


# create 4 modules using partials of slow_fun
# the 'module' arg to Vfunc.__init__ must either have a .fit method or be callable
modules = [Vfunc(f'fun{i}', partial(slow_fun, b=i)) for i in range(4)]

slow_set = Vset('slow_set', modules)

# if there's only one arg, it comes wrapped in a list and I can't pass the result of init_args directly to .fit
args = init_args([1], ['a'])[0]

In [4]:
%% time

slow_set(args)

CPU times: user 229 ms, sys: 24.8 ms, total: 254 ms
Wall time: 4 s


{('a', 'slow_set_0'): 1,
 ('a', 'slow_set_1'): 2,
 ('a', 'slow_set_2'): 3,
 ('a', 'slow_set_3'): 4,
 '__prev__': <vflow.module_set.ModuleSet at 0x7f85105ee190>}

In [5]:
# 1st option to use parallelism is to just use is_async=True when initializing ModuleSet
fast_set = Vset('fast_set', modules, is_async=True)

In [6]:
%% time

fast_set(args)

CPU times: user 96.1 ms, sys: 31.8 ms, total: 128 ms
Wall time: 1.12 s


{('a', 'fast_set_0'): 1,
 ('a', 'fast_set_1'): 2,
 ('a', 'fast_set_2'): 3,
 ('a', 'fast_set_3'): 4,
 '__prev__': <vflow.module_set.ModuleSet at 0x7f85105d6af0>}

In [7]:
# 2nd option is to use AsyncModules directly
async_modules = [AsyncModule(f'fun{i}', partial(slow_fun, b=i)) for i in range(4)]
fast_set = Vset('fast_set', async_modules)

In [8]:
%% time

fast_set(args)

CPU times: user 44 ms, sys: 20 ms, total: 64 ms
Wall time: 1.01 s


{('a', 'fast_set_0'): 1,
 ('a', 'fast_set_1'): 2,
 ('a', 'fast_set_2'): 3,
 ('a', 'fast_set_3'): 4,
 '__prev__': <vflow.module_set.ModuleSet at 0x7f85102a3d90>}